In [1]:
import pybullet as pyb
import time 
import pybullet_data
import numpy as np
import example_robot_data

import random 

In [2]:

mapJointLimits = [
    [-1.308996939, 1.308996939],        # TORSO 0-1
    [-0.261799387799, 0.785398163397],
    [-0.261799387799, 0.785398163397],  # HEAD 2-3
    [-1.308996939, 1.308996939],
    [-1.57079632679, 0.523598775598],   # LEFT ARM 4-10
    [0.0, 2.87979326579],
    [-2.44346095279, 2.44346095279],
    [-2.35619449019, 0.0],
    [-2.53072741539, 2.53072741539],
    [-1.3962634016, 1.3962634016],
    [-0.698131700798, 0.698131700798],
    [-0.523598775598, 1.57079632679], # RIGHT ARM 11-17
    [-2.87979326579, 0.0],
    [-2.44346095279, 2.44346095279],
    [-2.35619449019, 0.0],
    [-2.53072741539, 2.53072741539],
    [-1.3962634016, 1.3962634016],
    [-0.698131700798, 0.698131700798],
    [-0.349065850399, 1.57079632679],   # LEFT LEG 18-23
    [-0.5236, 0.5236],
    [-2.095, 0.7],
    [0.0, 2.618],
    [-1.309, 0.768],
    [-0.5236, 0.5236],
    [-1.57079632679, 0.349065850399],   # RIGHT LEG 24-29
    [-0.5236, 0.5236],
    [-2.095, 0.7],
    [0.0, 2.618],
    [-1.309, 0.768],
    [-0.5236, 0.5236]
]


gainsP = np.array([
    200.,  200.,                                        # TORSO 0-1
    6.0, 6.0,                                           # HEAD 2-3
    200.0, 400.,  100.,  100.,  10.,  10.,  10.,   # LEFT ARM 4-10
    200.,  400.,  100.,  100.,  10.,  10.,  10.,   # RIGHT ARM 11-17
    100.,  100.,  100.,  100.,  100.,  100.,          # LEFT LEG 18-23
    100.,  100.,  100.,  100.,  100.,  100.,          # RIGHT LEG 24-29
])
gainsD = np.array([
    10.,  10.,                          # TORSO 0-1
    0.1, 0.1,                           # HEAD 2-3
    0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,  # LEFT ARM 4-10
    0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,  # RIGHT ARM 11-17
    20.,  20.,  20.,  20.,  20.,  20.,  # LEFT LEG 18-23
    20.,  20.,  20.,  20.,  20.,  20.,  # RIGHT LEG 24-29
])


In [3]:

def m2a(m): return np.array(m.flat)

def a2m(a): return np.matrix(a).T

def getCurrentState(robotId, jointIndices):
    #print("JointIndices:",len(jointIndices)," -> ",jointIndices)
    jointStates = pyb.getJointStates(robotId, jointIndices) # State of all joints
    jointPositions = np.array([ value[0] for value in jointStates ])
    jointVelocities = np.array([ value[1] for value in jointStates ])
    return jointPositions, jointVelocities

def pdController(qdes, vdes):
    # Retrieve angular position and velocity of actuators
    #jointStates = pyb.getJointStates(robotId, controlledJoints)
    qmes, vmes = getCurrentState(robotId, controlledJoints)
    #print("len qmes:",len(qmes)," and qdes:",len(qdes))
    #print("len vmes:",len(vmes)," and vdes:",len(vdes))
    # Torque PD controller
    jointTorques = gainsP * (qdes - qmes) + gainsD * (vdes - vmes)
    return jointTorques

# ====================================== Main ====================================================

# Start the client for PyBullet
physicsClient = pyb.connect(pyb.GUI)
# Timestep is 1ms
dt = 1e-3
pyb.setTimeStep(dt)
# Set gravity (disabled by default)
pyb.setGravity(0,0,-9.81)

# Load horizontal plane
robotStartPosition = [0.0, 0.0,2]#1.08]
robotStartOrientation = pyb.getQuaternionFromEuler([0,0,0])

pyb.setAdditionalSearchPath(pybullet_data.getDataPath())
planeId = pyb.loadURDF("plane.urdf")
robotId = pyb.loadURDF("talos_reduced.urdf",robotStartPosition, robotStartOrientation, useFixedBase = True)

error: Cannot load URDF file.

In [4]:

#all joints 
bulletJointNames = [ pyb.getJointInfo(robotId,i)[1].decode() for i in range(pyb.getNumJoints(robotId)) ] # Name of all joints
JointIndicesComplete = [ i for i in range(0,len(bulletJointNames)) ]
JointIndicesComplete

NameError: name 'robotId' is not defined

In [5]:
nonControlledJoints = [
"imu_joint",                        # Other joints not used
"rgbd_joint",
"rgbd_optical_joint",
"rgbd_depth_joint",
"rgbd_depth_optical_joint",
"rgbd_rgb_joint",
"rgbd_rgb_optical_joint",
"wrist_left_ft_joint",              # RIGHT GRIPPER
"wrist_left_tool_joint",
"gripper_left_base_link_joint",
"gripper_left_joint",
"gripper_left_inner_double_joint",
"gripper_left_fingertip_1_joint",
"gripper_left_fingertip_2_joint",
"gripper_left_motor_single_joint",
"gripper_left_inner_single_joint",
"gripper_left_fingertip_3_joint",
"wrist_right_ft_joint",             # LEFT GRIPPER
"wrist_right_tool_joint",
"gripper_right_base_link_joint",
"gripper_right_joint",
"gripper_right_inner_double_joint",
"gripper_right_fingertip_1_joint",
"gripper_right_fingertip_2_joint",
"gripper_right_motor_single_joint",
"gripper_right_inner_single_joint",
"gripper_right_fingertip_3_joint",
"leg_left_sole_fix_joint",          # LEFT LEG (blocked)
"leg_right_sole_fix_joint"          # RIGHT LEG (blocked)
]

In [6]:
# Joints controlled (ID)
controlledJoints = [ i for (i,n) in enumerate(bulletJointNames) if n not in nonControlledJoints ]
#print("===================")
#print("List of joints controlled - ",len(controlledJoints)," joints :")
#for i in controlledJoints:
    #print(i," - ",bulletJointNames[i])
#print("===================")
print(controlledJoints[5])

12


In [7]:
# Get Lower/Upper limit of each joint controller (for PD controller)
controlledJointsBounds = []
for i in controlledJoints:
    info = pyb.getJointInfo(robotId ,i)
    controlledJointsBounds.append([info[8],info[9]])


In [8]:
len(controlledJointsBounds)

30

In [9]:
 def get_observation():
    observation = []
    for i in controlledJoints:
        joint_State = pyb.getJointState(robotId,i)
        observation.append(joint_State[:2])
    observation = np.array(observation)
    return observation


In [10]:
state = get_observation()
state

array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]])

In [11]:

# Reduced model : Joints controlled (Info)
rmodel =[pyb.getJointInfo(robotId,i) for i in controlledJoints ]

# Disable default position controler in torque controlled joints
# Default controller will take care of other joints
pyb.setJointMotorControlArray(robotId, 
                            jointIndices= controlledJoints, 
                            controlMode= pyb.VELOCITY_CONTROL,
                            targetVelocities=[0.0 for m in controlledJoints],
                            forces = [0.0 for m in controlledJoints]) 


In [12]:
rmodel


[(0,
  b'torso_1_joint',
  0,
  7,
  6,
  1,
  1.0,
  1.0,
  -1.308996939,
  1.308996939,
  78.0,
  5.4,
  b'torso_1_link',
  (0.0, 0.0, 1.0),
  (0.08222, -0.00838, 0.14481),
  (0.0, 0.0, 0.0, 1.0),
  -1),
 (1,
  b'torso_2_joint',
  0,
  8,
  7,
  1,
  1.0,
  1.0,
  -0.261799387799,
  0.785398163397,
  78.0,
  5.4,
  b'torso_2_link',
  (0.0, 1.0, 0.0),
  (-0.00013, 1e-05, 0.01306),
  (0.0, 0.0, 0.0, 1.0),
  0),
 (3,
  b'head_1_joint',
  0,
  9,
  8,
  1,
  0.5,
  1.0,
  -0.261799387799,
  0.785398163397,
  8.0,
  1.0,
  b'head_1_link',
  (0.0, 1.0, 0.0),
  (0.04551, 0.00053, 0.15214),
  (0.0, 0.0, 0.0, 1.0),
  1),
 (4,
  b'head_2_joint',
  0,
  10,
  9,
  1,
  0.5,
  1.0,
  -1.308996939,
  1.308996939,
  4.0,
  1.0,
  b'head_2_link',
  (0.0, 0.0, 1.0),
  (-0.0012, -0.00145, -0.02165),
  (0.0, 0.0, 0.0, 1.0),
  3),
 (11,
  b'arm_left_1_joint',
  0,
  11,
  10,
  1,
  1.0,
  1.0,
  -1.57079632679,
  0.523598775598,
  44.64,
  2.7,
  b'arm_left_1_link',
  (0.0, 0.0, 1.0),
  (0.04551, 0.15

In [13]:

# Enable torque control for revolute joints
jointTorques = [0.0 for m in controlledJoints]
pyb.setJointMotorControlArray(robotId, controlledJoints,
                              controlMode=pyb.TORQUE_CONTROL, forces=jointTorques)



In [14]:
q, vdes1 = getCurrentState(robotId, controlledJoints)
pre_qdes = q[::] 
len(pre_qdes)

30

In [ ]:

counterTest = 0
vdes = [0.0 for _ in controlledJoints]
q0, vdes = getCurrentState(robotId, controlledJoints)
while True:
    time.sleep(0.01)
    # Compute one step of simulation
    jointPositions, jointVelocities = getCurrentState(robotId, controlledJoints)

    # Pick qdes and vdes in : controlledJointsBounds
    print("CounterTest: ",counterTest)
    if counterTest<200:
        # Min joints limit
        #qdes = [ value[0] for value in mapJointLimits ]
        qdes = q0[::]
    else:
        # Min joints limit
        #qdes = [ value[1]/4 for value in mapJointLimits ]
        qdes = q0[::]
        qdes[5] = mapJointLimits[5][1]/2.0
        qdes[12] = mapJointLimits[11][1]/2.0
    counterTest+=1
    
    print("QD_Left = ", qdes[12], "QD_Right= ", qdes[5])
    torques = pdController(qdes, vdes)
    #torques = a2m(np.zeros(len(controlledJoints)))

    #Apply torque on complete model
    pyb.setJointMotorControlArray(robotId, controlledJoints, controlMode=pyb.TORQUE_CONTROL, forces=torques)
    pyb.stepSimulation()

In [15]:
len(mapJointLimits)

30

In [16]:
np.shape(mapJointLimits)

(30, 2)

In [17]:
import torch 

In [18]:
action = np.random.rand(30)
action 

array([0.47551037, 0.08243885, 0.79851955, 0.10365057, 0.29441488,
       0.89439413, 0.72891556, 0.73343389, 0.54198112, 0.32101668,
       0.73491872, 0.44602183, 0.38976047, 0.17077305, 0.39703569,
       0.97105686, 0.99530646, 0.97502062, 0.24181615, 0.15465305,
       0.34130552, 0.17425139, 0.48690146, 0.06876997, 0.10084775,
       0.2085481 , 0.1460783 , 0.80338864, 0.50139908, 0.92958055])

In [19]:
def apply_action():
    #Calculate Action state from normalization
    action = np.random.rand(30)
    action_Real = []
    for i in range (30):
        joint_Force = action [i] * (mapJointLimits[i][1]-mapJointLimits[i][0]) + mapJointLimits[i][0]
        action_Real.append(joint_Force)
    return action_Real

In [20]:
move_random = apply_action(action )

TypeError: apply_action() takes 0 positional arguments but 1 was given

In [21]:

counterTest = 0
vdes = [0.0 for _ in controlledJoints]
q0, vdes = getCurrentState(robotId, controlledJoints)
while True:
    time.sleep(0.01)
    # Compute one step of simulation
    jointPositions, jointVelocities = getCurrentState(robotId, controlledJoints)

    # Pick qdes and vdes in : controlledJointsBounds
    print("CounterTest: ",counterTest)
    if counterTest<200:
        # Min joints limit
        #qdes = [ value[0] for value in mapJointLimits ]
        qdes = q0[::]
    else:
        # Min joints limit
        #qdes = [ value[1]/4 for value in mapJointLimits ]
        qdes = apply_action()
        #qdes[5] = mapJointLimits[5][1]/2.0
        #qdes[12] = mapJointLimits[11][1]/2.0
    counterTest+=1
    
    print("QD_Left = ", qdes[12], "QD_Right= ", qdes[5])
    torques = pdController(qdes, vdes)
    #torques = a2m(np.zeros(len(controlledJoints)))

    #Apply torque on complete model
    pyb.setJointMotorControlArray(robotId, controlledJoints, controlMode=pyb.TORQUE_CONTROL, forces=torques)
    pyb.stepSimulation()

CounterTest:  0
QD_Left =  0.0 QD_Right=  0.0
CounterTest:  1
QD_Left =  0.0 QD_Right=  0.0
CounterTest:  2
QD_Left =  0.0 QD_Right=  0.0
CounterTest:  3
QD_Left =  0.0 QD_Right=  0.0
CounterTest:  4
QD_Left =  0.0 QD_Right=  0.0
CounterTest:  5
QD_Left =  0.0 QD_Right=  0.0
CounterTest:  6
QD_Left =  0.0 QD_Right=  0.0
CounterTest:  7
QD_Left =  0.0 QD_Right=  0.0
CounterTest:  8
QD_Left =  0.0 QD_Right=  0.0
CounterTest:  9
QD_Left =  0.0 QD_Right=  0.0
CounterTest:  10
QD_Left =  0.0 QD_Right=  0.0
CounterTest:  11
QD_Left =  0.0 QD_Right=  0.0
CounterTest:  12
QD_Left =  0.0 QD_Right=  0.0
CounterTest:  13
QD_Left =  0.0 QD_Right=  0.0
CounterTest:  14
QD_Left =  0.0 QD_Right=  0.0
CounterTest:  15
QD_Left =  0.0 QD_Right=  0.0
CounterTest:  16
QD_Left =  0.0 QD_Right=  0.0
CounterTest:  17
QD_Left =  0.0 QD_Right=  0.0
CounterTest:  18
QD_Left =  0.0 QD_Right=  0.0
CounterTest:  19
QD_Left =  0.0 QD_Right=  0.0
CounterTest:  20
QD_Left =  0.0 QD_Right=  0.0
CounterTest:  21
QD_Lef

CounterTest:  183
QD_Left =  0.0 QD_Right=  0.0
CounterTest:  184
QD_Left =  0.0 QD_Right=  0.0
CounterTest:  185
QD_Left =  0.0 QD_Right=  0.0
CounterTest:  186
QD_Left =  0.0 QD_Right=  0.0
CounterTest:  187
QD_Left =  0.0 QD_Right=  0.0
CounterTest:  188
QD_Left =  0.0 QD_Right=  0.0
CounterTest:  189
QD_Left =  0.0 QD_Right=  0.0
CounterTest:  190
QD_Left =  0.0 QD_Right=  0.0
CounterTest:  191
QD_Left =  0.0 QD_Right=  0.0
CounterTest:  192
QD_Left =  0.0 QD_Right=  0.0
CounterTest:  193
QD_Left =  0.0 QD_Right=  0.0
CounterTest:  194
QD_Left =  0.0 QD_Right=  0.0
CounterTest:  195
QD_Left =  0.0 QD_Right=  0.0
CounterTest:  196
QD_Left =  0.0 QD_Right=  0.0
CounterTest:  197
QD_Left =  0.0 QD_Right=  0.0
CounterTest:  198
QD_Left =  0.0 QD_Right=  0.0
CounterTest:  199
QD_Left =  0.0 QD_Right=  0.0
CounterTest:  200
QD_Left =  -1.0007343167096694 QD_Right=  1.293072260499239
CounterTest:  201
QD_Left =  -2.0770066600349253 QD_Right=  2.1731593039857535
CounterTest:  202
QD_Left =

CounterTest:  300
QD_Left =  -1.1127233267790557 QD_Right=  0.20269645297822336
CounterTest:  301
QD_Left =  -1.966445645394329 QD_Right=  0.013143294393346083
CounterTest:  302
QD_Left =  -2.2611433973232775 QD_Right=  0.18928639150345208
CounterTest:  303
QD_Left =  -2.348731119270905 QD_Right=  1.057577838186918
CounterTest:  304
QD_Left =  -2.795282350538266 QD_Right=  1.3730821530010822
CounterTest:  305
QD_Left =  -1.2749934426472072 QD_Right=  0.6375586742119437
CounterTest:  306
QD_Left =  -2.6597557441304627 QD_Right=  0.43358824840779486
CounterTest:  307
QD_Left =  -1.0951305530361157 QD_Right=  1.1400372268233034
CounterTest:  308
QD_Left =  -0.9729760963755978 QD_Right=  0.7899034671603101
CounterTest:  309
QD_Left =  -0.8760042053909447 QD_Right=  2.8060689130269467
CounterTest:  310
QD_Left =  -1.0990920207993056 QD_Right=  2.4890304190806174
CounterTest:  311
QD_Left =  -1.6057064321077559 QD_Right=  1.090611390822385
CounterTest:  312
QD_Left =  -1.2512058494836877 QD_

CounterTest:  412
QD_Left =  -0.744478179307714 QD_Right=  2.3466773227322077
CounterTest:  413
QD_Left =  -1.4796806314257447 QD_Right=  1.5302585210771382
CounterTest:  414
QD_Left =  -0.4029913949916568 QD_Right=  0.04966295886654052
CounterTest:  415
QD_Left =  -2.311438893484796 QD_Right=  2.0295367181319723
CounterTest:  416
QD_Left =  -2.7913513588608394 QD_Right=  1.7921209915050555
CounterTest:  417
QD_Left =  -0.42763599342064174 QD_Right=  2.1385625732398306
CounterTest:  418
QD_Left =  -0.9667377230967249 QD_Right=  1.5279845394737555
CounterTest:  419
QD_Left =  -0.47645170603587683 QD_Right=  2.1959410584829695
CounterTest:  420
QD_Left =  -2.0861641391638712 QD_Right=  0.617274376429
CounterTest:  421
QD_Left =  -0.2414870843476722 QD_Right=  2.5698017593053124
CounterTest:  422
QD_Left =  -0.2517080503732645 QD_Right=  1.645262950168165
CounterTest:  423
QD_Left =  -1.3320198388524074 QD_Right=  2.367161931787753
CounterTest:  424
QD_Left =  -1.1545954456014804 QD_Right

CounterTest:  516
QD_Left =  -2.5633681894686173 QD_Right=  1.1196070712465385
CounterTest:  517
QD_Left =  -1.9785247399441979 QD_Right=  0.4564358328125533
CounterTest:  518
QD_Left =  -2.152886662417351 QD_Right=  2.2780470140675106
CounterTest:  519
QD_Left =  -1.594138064297668 QD_Right=  1.933958078159405
CounterTest:  520
QD_Left =  -0.6367427140568993 QD_Right=  1.0655069269073656
CounterTest:  521
QD_Left =  -0.8883209374352234 QD_Right=  0.6541551266028793
CounterTest:  522
QD_Left =  -2.7533554124088138 QD_Right=  1.8202140653491898
CounterTest:  523
QD_Left =  -0.5195076072407856 QD_Right=  2.8648289090032013
CounterTest:  524
QD_Left =  -0.1344718513710217 QD_Right=  2.4387606604372616
CounterTest:  525
QD_Left =  -1.0443762309264308 QD_Right=  0.7195051612346357
CounterTest:  526
QD_Left =  -2.60875759647439 QD_Right=  1.108063805569802
CounterTest:  527
QD_Left =  -2.1051331548475885 QD_Right=  2.724131540035397
CounterTest:  528
QD_Left =  -2.3451460631032046 QD_Right= 

...

CounterTest:  18436
QD_Left =  -2.413148861903476 QD_Right=  0.22124728395403528
CounterTest:  18437
QD_Left =  -1.5636384452072738 QD_Right=  0.5055073098894796
CounterTest:  18438
QD_Left =  -0.47065776248422875 QD_Right=  0.07429662108100057
CounterTest:  18439
QD_Left =  -0.22388656033498222 QD_Right=  1.1697117653794735
CounterTest:  18440
QD_Left =  -0.11612413245898301 QD_Right=  1.0746082641850019
CounterTest:  18441
QD_Left =  -1.0480749224364283 QD_Right=  0.7417681719615602
CounterTest:  18442
QD_Left =  -1.173391021505914 QD_Right=  1.7070914123972012
CounterTest:  18443
QD_Left =  -2.280521662316281 QD_Right=  2.1483554604422244
CounterTest:  18444
QD_Left =  -0.48945814545951993 QD_Right=  0.1304573592610077
CounterTest:  18445
QD_Left =  -2.47151583808463 QD_Right=  2.0520236648404873
CounterTest:  18446
QD_Left =  -0.5326093950198882 QD_Right=  0.7292418592779055
CounterTest:  18447
QD_Left =  -1.7648629962966391 QD_Right=  1.5337069495394158
CounterTest:  18448
QD_Left

CounterTest:  18539
QD_Left =  -1.7331851544124517 QD_Right=  0.7122125997622598
CounterTest:  18540
QD_Left =  -2.229124767991485 QD_Right=  2.3323998823843874
CounterTest:  18541
QD_Left =  -2.653828798833274 QD_Right=  1.9257668413423568
CounterTest:  18542
QD_Left =  -0.5253441293414052 QD_Right=  2.5447432743791665
CounterTest:  18543
QD_Left =  -1.5952252806518898 QD_Right=  2.264779130632699
CounterTest:  18544
QD_Left =  -1.2182054567186542 QD_Right=  1.7416510066562416
CounterTest:  18545
QD_Left =  -0.6468315890432659 QD_Right=  0.0860109260226841
CounterTest:  18546
QD_Left =  -2.2071493812896206 QD_Right=  1.3308130262525648
CounterTest:  18547
QD_Left =  -1.7658928038767925 QD_Right=  0.33350704529179714
CounterTest:  18548
QD_Left =  -1.3994673202058223 QD_Right=  2.0015120757450324
CounterTest:  18549
QD_Left =  -0.6279389450214379 QD_Right=  1.8843363464116407
CounterTest:  18550
QD_Left =  -0.18506551018963213 QD_Right=  1.398805101732885
CounterTest:  18551
QD_Left = 

CounterTest:  18642
QD_Left =  -0.15429603825878146 QD_Right=  0.3874765068413053
CounterTest:  18643
QD_Left =  -1.7930079374891583 QD_Right=  1.2685994916539134
CounterTest:  18644
QD_Left =  -0.4201248981459318 QD_Right=  0.09129522328418492
CounterTest:  18645
QD_Left =  -1.4869305066846843 QD_Right=  2.028945197738946
CounterTest:  18646
QD_Left =  -1.4819511261995346 QD_Right=  2.0135680104288562
CounterTest:  18647
QD_Left =  -1.3632665632849077 QD_Right=  2.1045539205290082
CounterTest:  18648
QD_Left =  -2.866697318247575 QD_Right=  1.0916902202242964
CounterTest:  18649
QD_Left =  -0.6591069528988474 QD_Right=  0.6584726599179546
CounterTest:  18650
QD_Left =  -2.313226947445302 QD_Right=  2.2858089775896544
CounterTest:  18651
QD_Left =  -1.1815727012250674 QD_Right=  2.2720340555994287
CounterTest:  18652
QD_Left =  -0.051694060902682804 QD_Right=  0.01021021700288857
CounterTest:  18653
QD_Left =  -1.9229928534208338 QD_Right=  0.28197923480420706
CounterTest:  18654
QD_Le

CounterTest:  18754
QD_Left =  -1.1778617678951224 QD_Right=  2.0779383556874866
CounterTest:  18755
QD_Left =  -0.7351634754768996 QD_Right=  0.8177168939944552
CounterTest:  18756
QD_Left =  -2.815268981425358 QD_Right=  0.8534382409365086
CounterTest:  18757
QD_Left =  -0.6198155551566016 QD_Right=  1.2477213747317464
CounterTest:  18758
QD_Left =  -2.0662646703833785 QD_Right=  0.8719010636730713
CounterTest:  18759
QD_Left =  -0.1558742637342556 QD_Right=  1.3806550814789413
CounterTest:  18760
QD_Left =  -0.6863108285092934 QD_Right=  2.050919692113035
CounterTest:  18761
QD_Left =  -1.5307910228039954 QD_Right=  1.8331936198938406
CounterTest:  18762
QD_Left =  -1.1056909150000205 QD_Right=  0.807496256455167
CounterTest:  18763
QD_Left =  -1.2867729899652136 QD_Right=  0.2839903323694732
CounterTest:  18764
QD_Left =  -0.6198039358402023 QD_Right=  1.9633533098640488
CounterTest:  18765
QD_Left =  -1.4672768497882396 QD_Right=  0.32695648365734314
CounterTest:  18766
QD_Left = 

KeyboardInterrupt: 